In [1]:
!pip install geopandas
!pip install matplotlib
!pip install folium matplotlib mapclassify

  Using cached Fiona-1.8.21.tar.gz (1.0 MB)
  Using cached Fiona-1.8.20.tar.gz (1.3 MB)
  Using cached Fiona-1.8.19.tar.gz (1.3 MB)
  Using cached Fiona-1.8.18.tar.gz (1.3 MB)
  Using cached Fiona-1.8.17.tar.gz (1.3 MB)
  Using cached Fiona-1.8.16.tar.gz (1.3 MB)
  Using cached Fiona-1.8.15.tar.gz (1.3 MB)
  Using cached Fiona-1.8.14.tar.gz (1.3 MB)
  Using cached Fiona-1.8.13.post1.tar.gz (1.2 MB)
  Using cached Fiona-1.8.13.tar.gz (1.2 MB)
  Using cached Fiona-1.8.12.tar.gz (1.2 MB)
  Using cached Fiona-1.8.11.tar.gz (1.2 MB)
  Using cached Fiona-1.8.10.tar.gz (1.2 MB)
  Using cached Fiona-1.8.9.post2.tar.gz (1.2 MB)
  Using cached Fiona-1.8.9.post1.tar.gz (1.2 MB)
  Using cached Fiona-1.8.9.tar.gz (1.2 MB)
  Using cached Fiona-1.8.8.tar.gz (1.7 MB)
  Using cached Fiona-1.8.7.tar.gz (1.7 MB)
  Using cached Fiona-1.8.6.tar.gz (1.7 MB)
  Using cached Fiona-1.8.5.tar.gz (1.7 MB)
  Using cached Fiona-1.8.4.tar.gz (1.1 MB)
  Using cached Fiona-1.8.3.tar.gz (1.1 MB)
  Using cached Fiona-1.

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\davib\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\davib\\AppData\\Local\\Temp\\pip-install-wiw11evy\\fiona_a9725bddd4e04cee9a675432e6086831\\setup.py'"'"'; __file__='"'"'C:\\Users\\davib\\AppData\\Local\\Temp\\pip-install-wiw11evy\\fiona_a9725bddd4e04cee9a675432e6086831\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\davib\AppData\Local\Temp\pip-pip-egg-info-pna9q72l'
         cwd: C:\Users\davib\AppData\Local\Temp\pip-install-wiw11evy\fiona_a9725bddd4e04cee9a675432e6086831\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment 


  Using cached Fiona-1.7.13.tar.gz (731 kB)
  Running setup.py clean for fiona
Failed to build fiona
    Running setup.py install for fiona: started
    Running setup.py install for fiona: finished with status 'error'


    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\davib\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\davib\\AppData\\Local\\Temp\\pip-install-wiw11evy\\fiona_0bb5ee67e0934703b1104db606b91577\\setup.py'"'"'; __file__='"'"'C:\\Users\\davib\\AppData\\Local\\Temp\\pip-install-wiw11evy\\fiona_0bb5ee67e0934703b1104db606b91577\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\davib\AppData\Local\Temp\pip-pip-egg-info-1qt_41yb'
         cwd: C:\Users\davib\AppData\Local

In [ ]:
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import calendar


data = pd.read_csv('/content/drive/MyDrive/CameraViolations/Red_Light_Camera_Violations.csv').fillna(0)
data['Weekday'] = pd.to_datetime(data['VIOLATION DATE'])
data['Weekday'] = data['Weekday'].dt.weekday

data.describe() 
data.info()

# 783053 data rows

In [ ]:
print(data.isna().sum())
print(data.isnull().sum())
print(data['VIOLATIONS'].sum())
#there are no null or nan values in each column 

In [ ]:
data['VIOLATION DATE'].min()
# 01/01/2015
data['VIOLATION DATE'].max()
# 12/31/2021
year = [ el.split('/')[2] for el in data['VIOLATION DATE']]
month = [ el.split('/')[0] for el in data['VIOLATION DATE']]
day = [ el.split('/')[1] for el in data['VIOLATION DATE']]

data['Year'] = year
data['Month'] = month
data['Day'] = day

#drop the violation date as column because we added the year,month and day one
#drop the location because is a repetition of the latitude and longitude
data = data.drop(['VIOLATION DATE','LOCATION'],axis=1)

In [ ]:
print((data['X COORDINATE'] != 0).sum())
# 40921 violations have the relative position, for this reason we exclude the data with no geographical reference

data = data.loc[data['X COORDINATE'] != 0,:]

In [ ]:
violations_per_camera = data.groupby('CAMERA ID').sum().sort_values(by='VIOLATIONS',ascending=False)
violations_per_camera = violations_per_camera.iloc[:,0]
print(violations_per_camera)
l=[]
for el in data.loc[:,'CAMERA ID']: 
    l.append(violations_per_camera.loc[el])
data['TotalViolations'] = l

In [ ]:
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(data['LONGITUDE'],data['LATITUDE'])]

In [ ]:
street_map = gpd.read_file('/content/drive/MyDrive/CameraViolations/Boundaries - ZIP Codes.geojson')
gdf = gpd.GeoDataFrame(data, crs={'init':'epsg:4326'}, geometry=geometry)

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
street_map.to_crs(epsg=4326).plot(ax=ax, color='lightgrey')
gdf.plot(ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
street_map.to_crs(epsg=4326).plot(ax=ax, color='lightgrey')
gdf.plot('TotalViolations',ax=ax,legend=True)


In [ ]:
#gdf.explore()
fig, ax = plt.subplots(figsize = (15,15))
street_map.to_crs(epsg=4326).plot(ax=ax, color='#27285C')
gdf.plot('TotalViolations',ax=ax,cmap='OrRd',markersize=80, legend=True)
plt.title(' Total Violations for the years 2019 - 2021')


In [ ]:
ZeroDivisionError#Suppose 
data2019 = data[data['Year'] =='2019']
data2020 = data[data['Year'] =='2020']
data2021 = data[data['Year'] =='2021']

violations_per_camera_2019 = data2019.groupby(['Month']).sum().sort_values(by='Month',ascending=True)
violations_per_camera_2020 = data2020.groupby(['Month']).sum().sort_values(by='Month',ascending=True)
violations_per_camera_2021 = data2021.groupby(['Month']).sum().sort_values(by='Month',ascending=True)



In [ ]:
print(violations_per_camera_2020)

In [ ]:
geometry19 = [Point(xy) for xy in zip(data2019['LONGITUDE'],data2019['LATITUDE'])]
gdf2019 = gpd.GeoDataFrame(data2019, crs={'init':'epsg:4326'}, geometry=geometry19)
fig, ax = plt.subplots(figsize = (15,15))
street_map.to_crs(epsg=4326).plot(ax=ax, color='#27285C')
gdf2019.plot('TotalViolations',ax=ax,cmap='OrRd',markersize=80, legend=True)
plt.title(' Total Violations for the year of 2019')

In [ ]:
geometry20 = [Point(xy) for xy in zip(data2020['LONGITUDE'],data2020['LATITUDE'])]
gdf2020 = gpd.GeoDataFrame(data2020, crs={'init':'epsg:4326'}, geometry=geometry20)
fig, ax = plt.subplots(figsize = (15,15))
street_map.to_crs(epsg=4326).plot(ax=ax, color='#27285C')
gdf2020.plot('TotalViolations',ax=ax,cmap='OrRd',markersize=80, legend=True)
plt.title(' Total Violations for the year of 2020')

In [ ]:
geometry21 = [Point(xy) for xy in zip(data2021['LONGITUDE'],data2021['LATITUDE'])]
gdf2021 = gpd.GeoDataFrame(data2021, crs={'init':'epsg:4326'}, geometry=geometry21)
fig, ax = plt.subplots(figsize = (15,15))
street_map.to_crs(epsg=4326).plot(ax=ax, color='#27285C')
gdf2021.plot('TotalViolations',ax=ax,cmap='OrRd',markersize=80,legend=True)
plt.title(' Total Violations for the year of 2021')

In [ ]:
months = ['jan','feb','march','apr','may','june','july','aug','sept','oct','nov','dec']
plt.plot(months,violations_per_camera_2019['VIOLATIONS'])
plt.plot(months,violations_per_camera_2020['VIOLATIONS'])
plt.plot(months,violations_per_camera_2021['VIOLATIONS'])
plt.legend(['2019','2020','2021'])

In [ ]:
#Suppose 
#data2019_2021 = data
violations_per_camera_DOW = data.groupby(['Weekday']).sum().sort_values(by='Weekday',ascending=True)
print(violations_per_camera_DOW)

In [ ]:
days = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
plt.bar(days,violations_per_camera_DOW['VIOLATIONS'], color='#27285C')
plt.title("Comparison between the days of the week")

In [ ]:
#Bottom of Y limit scaled up to 50000 to show the difference
days = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
plt.bar(days,violations_per_camera_DOW['VIOLATIONS'], color='#27285C')
plt.title("Comparison between the days of the week")
plt.ylim(bottom=500000)

In [ ]:
#seasons = [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1]
Seas = ['Winter', 'Spring', 'Summer', 'Fall']
data['Season'] = (data['Month'].astype(int))%12 // 3 + 1

In [ ]:
violations_per_camera_seasons_of_19_21 = data.groupby(['Season']).sum().sort_values(by='Season',ascending=True)
plt.bar(Seas,violations_per_camera_seasons_of_19_21['VIOLATIONS'], color='#27285C')
plt.title("Comparison between the Seasons")
print(violations_per_camera_seasons_of_19_21)